In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.neighbors import NearestNeighbors
from sklearn.feature_extraction.text import TfidfVectorizer
import joblib

In [2]:
dataurl = 'https://docs.google.com/spreadsheets/d/1EmeVCJzCiMISggPuS5clv-gz4mznq24NMyFV8aXkbK4/edit?gid=0#gid=0'
dataurl = dataurl.replace('/edit?gid=', '/export?format=csv&gid=')
dataset = pd.read_csv(dataurl)
dataset = dataset.fillna('')
refDataset = dataset
dataset

,My Gender,My age,My city of residence,Video Introductions\n\n\n\nSubmit this if you wish to share yours https://forms.gle/bvNGD2CcixYQV7sYA,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else) the more detailed your response the more likely you will be asked for an introduction.","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc) the more detailed your response the more likely you will be asked for an introduction.",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist) the more detailed your response the more likely you will be asked for an introduction.",...,Anything else that you may like to mention to the community members (any specifics which may be important to you) the more detailed your response the more likely you will be asked for an introduction.,"Submitting this document means, I have shared factual information + giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.",My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Look for your row number in the excel sheet once you have submitted this doc. Your row# on submitting this doc will be found here,Referred by - share row# I name I city (this field is not mandatory - those who are referred need to fill this),Video Introduction
0,Male,33,Mumbai,,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...",Legally Separated,No,Between 40-50 Lakhs,Finance Professional,...,Health concious due to health conditions in fa...,I give consent,,,,,,,,
1,Male,36,New Delhi,,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,...,I like playing sports,I give consent,,,,,,,,
2,Male,32,Bangalore,,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),...,No,I give consent,,,,,,,,
3,Female,33,Gurugram,,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,...,I take time to open up and quite curious,I give consent,,,,,,,,
4,Female,32,Mumbai,,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,...,I believe honest communication and mutual resp...,I give consent,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
463,Female,37,Delhi,,"Able to take your own decisions, being financi...","Could be better but could be worse too, but th...",Never married,No,Under 12 Lakhs,MNC employee,...,I am an artist by hobby and love travelling am...,I give consent,Hindu,Yes,Yes I am,I am not sure,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
464,Female,38,New delhi,,"Supportive family, friends and a well paid job",Ignorant adolescents,"Was married earlier, legally divorced",Yes,Under 12 Lakhs,Content writer,...,Walk away from the place where you aren’t valued,I give consent,Hindu,Yes,Yes I am,I am not sure,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
465,Female,38,Faridabad,,I view privilege as the profound gift of wakin...,"My childhood was like a dream, filled with the...","Was married earlier, legally divorced",No,Between 20-24 Lakhs,I am a PhD holder and currently serve as an As...,...,Those who know me well describe me as a deeply...,I give consent,Sikh,Yes,I am not sure,Yes,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
466,Male,32,New De

In [3]:
dataset.to_csv("dataset.csv")

In [4]:
dataset.index = np.arange(2, len(dataset) + 2)
dataset

,My Gender,My age,My city of residence,Video Introductions\n\n\n\nSubmit this if you wish to share yours https://forms.gle/bvNGD2CcixYQV7sYA,"I define privilege to be (having a supportive family, dependable friends, a job you love, anything else) the more detailed your response the more likely you will be asked for an introduction.","About my growing up years (they were awesome, they were challenging but all is good now, about the family, about friends, memories etc etc) the more detailed your response the more likely you will be asked for an introduction.",My current relationship status,I have children,Ball park of my professional annual income,"How do I introduce the professional me (banker, IT professional, entrepreneur, environmentalist) the more detailed your response the more likely you will be asked for an introduction.",...,Anything else that you may like to mention to the community members (any specifics which may be important to you) the more detailed your response the more likely you will be asked for an introduction.,"Submitting this document means, I have shared factual information + giving consent (barring your personal details - full name and contact number) for your details to be viewed by andwemet singles community members.",My religious inclination,I am open to meeting someone no more married,I am fine with meeting a single parent,Open to meeting someone form faith not similar to mine,How did you hear about the community,Look for your row number in the excel sheet once you have submitted this doc. Your row# on submitting this doc will be found here,Referred by - share row# I name I city (this field is not mandatory - those who are referred need to fill this),Video Introduction
2,Male,33,Mumbai,,Having things that money cant buy.,"Defense upbringing, kept moving cities and sch...",Legally Separated,No,Between 40-50 Lakhs,Finance Professional,...,Health concious due to health conditions in fa...,I give consent,,,,,,,,
3,Male,36,New Delhi,,"Family,friends",Awesome,Never married,No,Between 30-35 lakhs,Education institute owner,...,I like playing sports,I give consent,,,,,,,,
4,Male,32,Bangalore,,Supportive/understandable/Friendly,"Awesome & challenging, All Good Now.",Never married,No,Between 12-15 Lakhs,Engineer (Interior Design),...,No,I give consent,,,,,,,,
5,Female,33,Gurugram,,Being able to love people and be loved by them,We’re full of curiosity and learning I was an ...,Was married earlier,No,Between 18-20 Lakhs,Mental health professional,...,I take time to open up and quite curious,I give consent,,,,,,,,
6,Female,32,Mumbai,,Being able to choose and make a life for myself,"Coming from a humble background, it has taught...",Never married,No,Between 18-20 Lakhs,Producer in Advertising,...,I believe honest communication and mutual resp...,I give consent,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
465,Female,37,Delhi,,"Able to take your own decisions, being financi...","Could be better but could be worse too, but th...",Never married,No,Under 12 Lakhs,MNC employee,...,I am an artist by hobby and love travelling am...,I give consent,Hindu,Yes,Yes I am,I am not sure,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
466,Female,38,New delhi,,"Supportive family, friends and a well paid job",Ignorant adolescents,"Was married earlier, legally divorced",Yes,Under 12 Lakhs,Content writer,...,Walk away from the place where you aren’t valued,I give consent,Hindu,Yes,Yes I am,I am not sure,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
467,Female,38,Faridabad,,I view privilege as the profound gift of wakin...,"My childhood was like a dream, filled with the...","Was married earlier, legally divorced",No,Between 20-24 Lakhs,I am a PhD holder and currently serve as an As...,...,Those who know me well describe me as a deeply...,I give consent,Sikh,Yes,I am not sure,Yes,Instagram,https://docs.google.com/spreadsheets/d/1EmeVCJ...,,
468,Male,32,New De

In [5]:
dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,6])

In [6]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,33,Mumbai,Legally Separated
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
465,Female,37,Delhi,Never married
466,Female,38,New delhi,"Was married earlier, legally divorced"
467,Female,38,Faridabad,"Was married earlier, legally divorced"
468,Male,32,New Delhi,Never married


In [7]:
age_weight =2.0
relationship_weight = 1.5
city_weight = 1.0


In [8]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

In [9]:
# one hot encoding
encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

In [10]:
encoded_categorical_features

,Legally Separated,Never married,"Separated, filed for a divorce","Separated, not legally divorced",Was married earlier,"Was married earlier, legally divorced",Widow/ Widower
2,1,0,0,0,0,0,0
3,0,1,0,0,0,0,0
4,0,1,0,0,0,0,0
5,0,0,0,0,1,0,0
6,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...
465,0,1,0,0,0,0,0
466,0,0,0,0,0,1,0
467,0,0,0,0,0,1,0
468,0,1,0,0,0,0,0


In [11]:
#label encode gender

from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

In [12]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,33,Mumbai,Legally Separated
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
465,Female,37,Delhi,Never married
466,Female,38,New delhi,"Was married earlier, legally divorced"
467,Female,38,Faridabad,"Was married earlier, legally divorced"
468,Male,32,New Delhi,Never married


In [13]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

In [14]:
dataset

,My Gender,My age,My city of residence,My current relationship status
2,Male,33,Mumbai,Legally Separated
3,Male,36,New Delhi,Never married
4,Male,32,Bangalore,Never married
5,Female,33,Gurugram,Was married earlier
6,Female,32,Mumbai,Never married
...,...,...,...,...
465,Female,37,Delhi,Never married
466,Female,38,New delhi,"Was married earlier, legally divorced"
467,Female,38,Faridabad,"Was married earlier, legally divorced"
468,Male,32,New Delhi,Never married


In [15]:
labeledFeatures=labeledFeatures.reshape(-1,1)

print(city_vectorized.toarray())

[[0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 ...
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]
 [0. 0. 0. ... 0. 0. 0.]]


In [16]:
# prepare the data for training
features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized.toarray()])


In [17]:
print(features.shape)

(468, 163)


In [18]:
k =280
knn = NearestNeighbors(n_neighbors=k, metric='cosine')
knn.fit(features)

NearestNeighbors(metric='cosine', n_neighbors=280)

In [19]:
row =210
selectedRow = refDataset.iloc[row]

In [20]:
distances, indices = knn.kneighbors(features[row].reshape(1,-1))
print(indices)

[[210 220 113 401 109 252 409 424 463 396 316  11 108 460 372 272  45 266
  165 342 440  25 313 331 320  41 330 385 278 383 332 326 343 245 247 290
  261 156 351  48  24 311  81 229 386  26 425 358 309 139 452 449 181 256
  354 363 333 123 288 118  97  51 122  30 390  54 128 295 110 376 462 262
   86 136 412  84 454 349 414 260 381  19 231  75 259  55 325 274 446 336
  310 307 114 365 393 183  42 375 265 287 355 308 200  52 415 353   1  27
  174 100 378  32 105 441 456  50 447  18 404  73   6 457 185 327  29 228
   12  65 251 111 344 258 301 178 453 221 291 282 467 373 246 388  96 420
   66 166  77 121 296 435 406 443 345  83 201 348 244  56 135 380 237 196
  169 428  36 411 106 328 160 209 340 432 143  91  89 241 194 208 255 171
  163 219 335 357  71 379 421 107 321 352 230 464  76 184 314 438 104  21
  112 271 264 238  35 413 382 211 142 147 119 205 214 442 422 249 173 403
  408 263  53 444 242 141  94 191  70 137  79 391  62  23 315 180  59  37
  297 346 188 426 405 101 257   9 429 

In [21]:
for i in indices[0]:
    if(dataset.iloc[i]['My Gender'] == selectedRow["My Gender"]):
        continue
    print(dataset.iloc[i].values)
    print("")
    print("")


['Male' 43 'Delhi' 'Never married']


['Male' 47 'New Delhi ' 'Never married']


['Male' 42 'New Delhi ' 'Never married']


['Male' 55 'Dubai ' 'Never married']


['Male' 54 'Delhi ' 'Was married earlier']


['Male' 50 'Delhi ' 'Separated, not legally divorced']


['Male' 49 'Bangalore, Karnataka, India' 'Never married']


['Male' 49 'Pune' 'Never married']


['Male' 49 'Pune' 'Never married']


['Male' 45 'Delhi ' 'Was married earlier']


['Male' 45 'Mumbai ' 'Never married']


['Male' 37 'Delhi' 'Never married']


['Male' 44 'Delhi' 'Widow/ Widower']


['Male' 44 'Delhi' 'Was married earlier']


['Male' 49 'New Delhi ' 'Was married earlier']


['Male' 43 'Mumbai ' 'Never married']


['Male' 43 'Tiruvalla ' 'Never married']


['Male' 43 'Delhi' 'Was married earlier']


['Male' 36 'Delhi' 'Never married']


['Male' 36 'Delhi ' 'Never married']


['Male' 42 'Mumbai' 'Never married']


['Male' 42 'Bangalore ' 'Never married']


['Male' 38 'New Delhi' 'Never married']


['Male' 38 'New De

In [22]:
def preprocessData(dataset):
    dataset = (dataset.iloc[:,:3]).join(dataset.iloc[:,6])

    vectorizer = TfidfVectorizer()
    city_vectorized = vectorizer.fit_transform(dataset['My city of residence'])

    # one hot encoding
    encoded_categorical_features =  (pd.get_dummies(dataset['My current relationship status'], dtype=int))

    #label encode
    labelencoder = LabelEncoder()
    labeledFeatures = labelencoder.fit_transform(dataset['My Gender'])

    scaler = StandardScaler()
    scaled_numerical_features= scaler.fit_transform((dataset.iloc[:,1].values).reshape(-1,1))

    labeledFeatures=labeledFeatures.reshape(-1,1)
    city_vectorized = city_vectorized.toarray()

    features = np.hstack([labeledFeatures, scaled_numerical_features, encoded_categorical_features.values, city_vectorized])
    return features

In [23]:
print(preprocessData(refDataset))

[[ 1.         -0.73868015  1.         ...  0.          0.
   0.        ]
 [ 1.         -0.23375954  0.         ...  0.          0.
   0.        ]
 [ 1.         -0.90698701  0.         ...  0.          0.
   0.        ]
 ...
 [ 0.          0.1028542   0.         ...  0.          0.
   0.        ]
 [ 1.         -0.90698701  0.         ...  0.          0.
   0.        ]
 [ 0.         -0.06545267  0.         ...  0.          0.
   0.        ]]


In [24]:
joblib.dump(knn, 'simpleclassifier.joblib')

['simpleclassifier.joblib']